In [ ]:
"""
Extract Search Parameters from PRIDE Dataset PXD044513

Purpose: Extract database search parameters from .mzid and .msfView files
to document the exact settings used in the proteomics analysis.

Input files:
- NB_Jan2021_120mins_OTIT_A5_1.mzid
- NB_Jan2021_120mins_OTIT_A5_1.msfView

Date: February 12, 2026
"""

In [1]:
from pyteomics import mzid
import xml.etree.ElementTree as ET
import sqlite3
import pandas as pd

# Extract from .msfView Database

In [2]:
import sqlite3
import pandas as pd

# Connect to the Proteome Discoverer database
msf_file = "NB_Jan2021_120mins_OTIT_A5_1.msfView"

print("\n" + "=" * 60)
print("EXTRACTING PARAMETERS FROM PROTEOME DISCOVERER DATABASE")
print("=" * 60)

try:
    conn = sqlite3.connect(msf_file)
    
    # 1. Processing Node Information (Search Engine)
    print("\n1. PROCESSING WORKFLOW")
    print("-" * 40)
    try:
        nodes = pd.read_sql_query("""
            SELECT ProcessingNodeNumber, NodeName, FriendlyName 
            FROM ProcessingNodes
            WHERE NodeName LIKE '%Sequest%' 
               OR NodeName LIKE '%Mascot%'
               OR NodeName LIKE '%Search%'
               OR NodeName LIKE '%Percolator%'
            ORDER BY ProcessingNodeNumber
        """, conn)
        if len(nodes) > 0:
            print(nodes.to_string(index=False))
        else:
            print("   No search nodes found, showing all nodes:")
            all_nodes = pd.read_sql_query("""
                SELECT ProcessingNodeNumber, NodeName 
                FROM ProcessingNodes
                ORDER BY ProcessingNodeNumber
                LIMIT 20
            """, conn)
            print(all_nodes.to_string(index=False))
    except Exception as e:
        print(f"   Error: {e}")
    
    # 2. Search Parameters
    print("\n2. DETAILED SEARCH PARAMETERS")
    print("-" * 40)
    try:
        # Get parameters from search nodes
        params = pd.read_sql_query("""
            SELECT 
                pn.NodeName,
                pnp.ParameterName, 
                pnp.ParameterValue
            FROM ProcessingNodeParameters pnp
            JOIN ProcessingNodes pn ON pnp.ProcessingNodeNumber = pn.ProcessingNodeNumber
            ORDER BY pn.ProcessingNodeNumber, pnp.ParameterName
        """, conn)
        
        if len(params) > 0:
            # Show key parameters
            key_params = ['Enzyme', 'Database', 'Precursor', 'Fragment', 'Tolerance', 
                          'Missed', 'FDR', 'Modification', 'Carbamidomethyl', 'Oxidation']
            
            for keyword in key_params:
                matching = params[params['ParameterName'].str.contains(keyword, case=False, na=False) | 
                                 params['ParameterValue'].str.contains(keyword, case=False, na=False)]
                if len(matching) > 0:
                    print(f"\n   Parameters containing '{keyword}':")
                    for _, row in matching.iterrows():
                        print(f"   {row['ParameterName']}: {row['ParameterValue']}")
        else:
            print("   No parameters found")
    except Exception as e:
        print(f"   Error: {e}")
    
    # 3. Amino Acid Modifications
    print("\n3. AMINO ACID MODIFICATIONS")
    print("-" * 40)
    try:
        mods = pd.read_sql_query("""
            SELECT 
                ModificationName,
                Abbreviation,
                DeltaMass,
                Substitution,
                IsActive
            FROM AminoAcidModifications
            WHERE IsActive = 1
        """, conn)
        
        if len(mods) > 0:
            for _, mod in mods.iterrows():
                print(f"   {mod['ModificationName']} ({mod['Abbreviation']})")
                print(f"      Mass shift: {mod['DeltaMass']:.3f} Da")
                print(f"      Residue: {mod['Substitution']}")
        else:
            print("   No modifications found")
    except Exception as e:
        print(f"   Error: {e}")
    
    # 4. FASTA Files (Database)
    print("\n4. PROTEIN DATABASES")
    print("-" * 40)
    try:
        fasta = pd.read_sql_query("""
            SELECT 
                FileName,
                NumberOfProteins,
                NumberOfAminoAcids
            FROM FastaFiles
        """, conn)
        if len(fasta) > 0:
            print(fasta.to_string(index=False))
        else:
            print("   No FASTA file information found")
    except Exception as e:
        print(f"   Error: {e}")
    
    conn.close()
    
except Exception as e:
    print(f"\nError connecting to database: {e}")
    print("Make sure the .msfView file is in the same directory")

print("\n" + "=" * 60)
print("EXTRACTION COMPLETE")
print("=" * 60)


EXTRACTING PARAMETERS FROM PROTEOME DISCOVERER DATABASE

1. PROCESSING WORKFLOW
----------------------------------------
   Error: Execution failed on sql '
            SELECT ProcessingNodeNumber, NodeName, FriendlyName 
            FROM ProcessingNodes
            WHERE NodeName LIKE '%Sequest%' 
               OR NodeName LIKE '%Mascot%'
               OR NodeName LIKE '%Search%'
               OR NodeName LIKE '%Percolator%'
            ORDER BY ProcessingNodeNumber
        ': no such table: ProcessingNodes

2. DETAILED SEARCH PARAMETERS
----------------------------------------
   Error: Execution failed on sql '
            SELECT 
                pn.NodeName,
                pnp.ParameterName, 
                pnp.ParameterValue
            FROM ProcessingNodeParameters pnp
            JOIN ProcessingNodes pn ON pnp.ProcessingNodeNumber = pn.ProcessingNodeNumber
            ORDER BY pn.ProcessingNodeNumber, pnp.ParameterName
        ': no such table: ProcessingNodeParameters

3.

In [3]:
import sqlite3
import pandas as pd

msf_file = "NB_Jan2021_120mins_OTIT_A5_1.msfView"

print("=" * 60)
print("DISCOVERING DATABASE STRUCTURE")
print("=" * 60)

try:
    conn = sqlite3.connect(msf_file)
    
    # 1. List ALL tables
    print("\n1. ALL TABLES IN DATABASE")
    print("-" * 40)
    tables = pd.read_sql_query("""
        SELECT name, type 
        FROM sqlite_master 
        WHERE type IN ('table', 'view')
        ORDER BY name
    """, conn)
    print(tables.to_string(index=False))
    print(f"\nTotal tables/views: {len(tables)}")
    
    # 2. For each table, show structure
    print("\n2. TABLE STRUCTURES (first 3 rows of each)")
    print("-" * 40)
    
    for table_name in tables['name'].head(20):  # Show first 20 tables
        try:
            print(f"\n--- Table: {table_name} ---")
            # Get column info
            cols = pd.read_sql_query(f"PRAGMA table_info({table_name})", conn)
            print("Columns:", ", ".join(cols['name'].tolist()))
            
            # Get sample data
            sample = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 3", conn)
            if len(sample) > 0:
                print(f"Sample data ({len(sample)} rows):")
                print(sample.to_string(index=False))
            else:
                print("(Empty table)")
        except Exception as e:
            print(f"Error reading {table_name}: {e}")
    
    conn.close()
    
except Exception as e:
    print(f"Error: {e}")

print("\n" + "=" * 60)
print("COMPLETE - Now we know what tables exist!")
print("=" * 60)

DISCOVERING DATABASE STRUCTURE

1. ALL TABLES IN DATABASE
----------------------------------------
                                          name  type
                     DataTypeAdditionalColumns table
                         DataTypeSortedSubsets table
                         DataTypeSubsetDetails table
                               DataTypeSubsets table
                        DecoyProteins_Subset_1 table
                            DecoyPsms_Subset_1 table
                            DecoyPsms_Subset_2 table
                            DecoyPsms_Subset_3 table
 DecoyPsms_Subset_3_WorkflowIDAsc_PeptideIDAsc table
          FoundModifications_ModificationIDAsc table
   MSnSpectrumInfo_WorkflowIDAsc_SpectrumIDAsc table
                                    ReportInfo table
                                    SchemaInfo table
                                 SettingsOwner table
                                SettingsValues table
                       TargetProteins_Subset_1 table


In [4]:
import sqlite3
import pandas as pd

msf_file = "NB_Jan2021_120mins_OTIT_A5_1.msfView"
conn = sqlite3.connect(msf_file)

print("=" * 60)
print("EXTRACTING AVAILABLE INFORMATION FROM .msfView FILE")
print("=" * 60)

# 1. Software version and date
print("\n1. SOFTWARE & ANALYSIS INFO")
print("-" * 40)
schema = pd.read_sql_query("SELECT * FROM SchemaInfo", conn)
print(schema.to_string(index=False))

# 2. Report info
print("\n2. ORIGINAL FILE INFO")
print("-" * 40)
report = pd.read_sql_query("SELECT * FROM ReportInfo", conn)
print(f"Original file: {report['ReportFileName'].values[0]}")
print(f"Report GUID: {report['ReportGuid'].values[0]}")

# 3. PSM Statistics
print("\n3. PSM STATISTICS (From this file)")
print("-" * 40)
subsets = pd.read_sql_query("SELECT * FROM DataTypeSubsetDetails", conn)
for _, row in subsets.iterrows():
    print(f"{row['TableName']}: {row['ItemCount']} items")

# 4. Check what modifications were found
print("\n4. MODIFICATIONS DETECTED IN DATA")
print("-" * 40)
mods = pd.read_sql_query("SELECT * FROM FoundModifications_ModificationIDAsc LIMIT 10", conn)
print(f"Number of modification IDs: {len(mods)}")
print("Modification IDs found:", mods['ModificationID'].tolist())

# 5. Workflow info
print("\n5. WORKFLOW INFORMATION")
print("-" * 40)
workflows = pd.read_sql_query("SELECT DISTINCT WorkflowID FROM TargetPsms_Subset_1", conn)
print(f"Workflow ID: {workflows['WorkflowID'].values[0]}")

conn.close()

print("\n" + "=" * 60)
print("CONCLUSION:")
print("This .msfView file is a REPORT VIEW (filtered results only)")
print("Search parameters are in the original .msf file:")
print("  NB_Jan2021_120mins_OTIT_A5_1.msf")
print("\nTo get full parameters, you need to download the .msf file")
print("from PRIDE, not the .msfView file.")
print("=" * 60)

EXTRACTING AVAILABLE INFORMATION FROM .msfView FILE

1. SOFTWARE & ANALYSIS INFO
----------------------------------------
 Version             Kind                        Date SoftwareVersion Comment
       3       ReportView 2023-06-21 13:37:29.8340514       2.2.0.388 Created
       1 KeyValueSettings 2023-06-21 13:37:29.8340514       2.2.0.388 Created

2. ORIGINAL FILE INFO
----------------------------------------
Original file: G:\NiamhB_June2023\MS\All Files\NB_Jan2021_120mins_OTIT_A5_1.msf
Report GUID: b7fbc0b2-1f14-4fd3-9c8b-761d30bcb14c

3. PSM STATISTICS (From this file)
----------------------------------------
TargetPsms_Subset_1: 105286 items
DecoyPsms_Subset_1: 124878 items
TargetPsms_Subset_2: 105286 items
TargetProteins_Subset_1: 20132 items
DecoyPsms_Subset_2: 124878 items
DecoyProteins_Subset_1: 22001 items
TargetPsms_Subset_3: 105286 items
DecoyPsms_Subset_3: 124878 items

4. MODIFICATIONS DETECTED IN DATA
----------------------------------------
Number of modification 